# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링 (목표 user 1000명)

- user_profile_df : User의 아이디, 리뷰수, 팔로워, url을 저장함
- user_df : '아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수' 를 저장함


In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.action_chains import ActionChains

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  

from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import os
from datetime import datetime

import pandas as pd
from selenium.webdriver.common.by import By

### 함수 

In [2]:
# 정규 표현식으로 이모티콘, 특수문자, 아스키코드 제거
def remove_special_characters(text):
    pattern = r'[^\w\s]|_'
    result = re.sub(pattern, '', text)
    return result

In [3]:
# User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 

def user_profile(url) :
    # 주소 이동

    global driver
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    user_element = {}

    # User ID 찾기
    user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
    user_element['아이디'] = user_id_element.text if user_id_element else None

    # User의 인기도 : 리뷰수, 팔로워 찾기 
    user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
    
    for element in user_popularity:
        em_tag = element.find('em')
        if em_tag:
            key = element.text.replace(em_tag.text, '').strip()
            value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
            user_element[key] = value
    
    # User URL
    user_element['url'] = url

    print(user_element)

    return user_element

def change_date_format(날짜) :
    #  부분 추출 (년월일까지)
    date_part = ""
    for i in 날짜 :
        date_part += i
        date_part += ','
    date_part = date_part.split('\n')[1]

    # 일자, 요일, 방문일수 분류하기 
    date_part = date_part.split(',')

    # '21.1.3.'에서 숫자 부분 추출
    revisit_match = re.search(r'(\d+\.\d+\.\d+)\.', date_part[1])
    if revisit_match:
        revisit_str = revisit_match.group(1)
        # 방문일수 정수로 변환
        revisit = int(revisit_str.replace('.', ''))
    else:
        revisit = 0  # 또는 다른 기본값으로 설정

    # 일자 타입 변경하기 
    date = date_part[0]
    weekday = date[-3:]
    match = re.findall(r'(\d+)년 (\d+)월 (\d+)일', date) # 타입 바꾸기 
    
    if match:
        year, month, day = map(int, match[0])

        # 날짜 객체로 변환
        date_object = datetime(2000 + year, month, day) 
        formatted_date = date_object.strftime('%Y-%m-%d')
    else:
        formatted_date = ''  # 또는 다른 기본값으로 설정
    
    return formatted_date, weekday, revisit


In [4]:
# "내용 더보기" 텍스트를 포함하는 버튼 찾기


def click_read_more_button(driver):
    buttons = driver.find_elements(By.XPATH, '//button[contains(.,"내용 더보기")]')
    if buttons:
        first_button = buttons[0]
        first_button.click()
        time.sleep(1)

# 태그 더보기를 클릭 
def click_read_more_tag(driver):
    buttons = driver.find_elements(By.XPATH, '//span[contains(., "개의 리뷰 더 있습니다")]')
    if buttons:
        first_button = buttons[0]
        first_button.click()
        time.sleep(1)

In [5]:
# 유저 리뷰 가져오는 함수 
def find_user_data(page_down,driver) : 

 
    # data를 담을 빈 리스트 선언
    data_list = [] 

    # 버튼을 클래스로 찾아 리스트로 가져오기
    buttons = driver.find_elements(By.CLASS_NAME, '_3P-5HQ')
    if buttons:
        first_button = buttons[0]
        first_button.click()

    # 맨 처음 게시물 클릭 
    button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
    button.click()
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(driver.page_source, 'html.parser')


    # 페이지 아래로 내리기 
    for _ in range(page_down):

        # '내용더보기' 버튼들 찾고 죄다 클릭해두기  
        try :
            click_read_more_button(driver)
            click_read_more_tag(driver)
        except :
            pass 
                
        # 다음 페이지로 잘 넘어갔는지 기다리기 
        wait = WebDriverWait(driver, 10)
        user_review_elements = wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92')))
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 


        # 리뷰에서 정보 가져오기
        for reviews_elements in user_review_elements:
            i = 0 # 카테고리와 주소 구분할 때 사용
            restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')

            if bool(restaurant_elements): # 정보가 없는 경우가 있음

                # 매장명 찾기
                restaurant_name = restaurant_elements[0].text  

        #<--------------------------------------------------------------------------------------------------->
                # 카테고리와 주소 찾기 
                category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
                i = 0  # i 초기화
                for category_location_element in category_location_elements:
                    category_location_soup = BeautifulSoup(category_location_element.get_attribute('outerHTML'), 'html.parser')
                
                    # span 태그 안에 있는 텍스트 가져오기
                    span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
                    category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
                    location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
                #<--------------------------------------------------------------------------------------------------->

                    review_elements = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')
                    try : 
                        review_text = review_elements[0].text # 리뷰 본문      
                        review_text = remove_special_characters(review_text)
                        sub_info_elements = reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') # 세부정보
                        sub_info = sub_info_elements[0].text if sub_info_elements else None
                    except :
                        review_text = '리뷰 없음'
                        sub_info = ''
                #<--------------------------------------------------------------------------------------------------->
                    # tag 찾기 
                    reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b')  # 리엑션 리스트
                    command = False

                    for x in reactions_elements:
                        if x.get_attribute("role"):  # 리액션 숨김 없애기
                            try:
                                # 요소가 클릭 가능할 때까지 대기
                                wait = WebDriverWait(driver, 10)
                                reaction_element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'COw42b')))
                                
                                # JavaScript를 사용하여 요소 클릭
                                driver.execute_script("arguments[0].click();", reaction_element)
                                
                                command = True
                            except:
                                continue

                    if command:
                        # 리엑션 리스트 재탐색
                        reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b')

                    # 리스트를 벗김
                    reactions = list(map(lambda x: x.text, reactions_elements))
                    reactions = str(reactions).replace('[', '').replace(']', '')                    

                #<--------------------------------------------------------------------------------------------------->
                    # 방문 일자, 재방문 이력 찾기
                    date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
                    date=[x.text for x in date_info]
                    day,weekday,revisit = change_date_format(date) # 함수 사용

                    global user_data

                    # 정보 추가
                    data_dict = {
                        '아이디' : user_data['아이디'],
                        '매장명': restaurant_name,
                        '카테고리': category,
                        '주소' : location,
                        '리뷰 내용': review_text,
                        "세부정보" : sub_info,
                        '태그': reactions,
                        '방문일자': day,
                        '요일' : weekday,
                        '재방문횟수':revisit
                    }
                    data_list.append(data_dict)

        user_df = pd.DataFrame(data_list)
        user_df = user_df.drop_duplicates()

    return user_df



In [6]:
url_list = pd.read_excel(r'C:\Users\hanjieun_2\Desktop\data\crawling_data\data\4_1_naver_user_크롤링_500개이상.xlsx')
url_list = url_list[url_list['담당']=='지은'].drop_duplicates(['아이디'])
url_list

,검색어,아이디,url,담당
250,"압구정로79길 32, 청담동마녀김밥",SunnyDay15,https://m.place.naver.com/my/5be31b2e19faf1454...,지은
252,"압구정로79길 32, 청담동마녀김밥",stay real,https://m.place.naver.com/my/5dd12f9c8f87a842b...,지은
253,"압구정로79길 32, 청담동마녀김밥",스트로베리홍시,https://m.place.naver.com/my/5e8f2ffe8f87a842b...,지은
256,"압구정로79길 32, 청담동마녀김밥",5430583,https://m.place.naver.com/my/5ecf276d8f87a842b...,지은
257,"압구정로79길 32, 청담동마녀김밥",외계인의별,https://m.place.naver.com/my/5dc4deb18f87a842b...,지은
...,...,...,...,...
495,"봉은사로4길 37, 꼭그닭",존까뚱,https://m.place.naver.com/my/5e0762568f87a842b...,지은
496,"봉은사로4길 21, 무월",김쥴리,https://m.place.naver.com/my/5efde969854ca9293...,지은
497,"봉은사로4길 21, 무월",리뷰꾸르잼,https://m.place.naver.com/my/5d8c8a868f87a842b...,지은
498,"봉은사로4길 21, 무월",달밤에춤,https://m.place.naver.com/my/5d7380bc8f87a842b...,지은


### url에 들어가서 유저 정보와 리뷰 저장

In [7]:

# 해민 
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
import chromedriver_autoinstaller

In [8]:
options = webdriver.ChromeOptions()
options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# 새로운 창이 뜨지 않도록 설정
options.add_argument("--disable-popup-blocking")
options.add_experimental_option("excludeSwitches", ["enable-popup-blocking"])

In [9]:
start_num = 100 # 할 차례, 여기를 계속 수정해주세요. 
num =  29 # 몇개씩 크롤링할지(기본은 100개를 추천합니다. -> 16분)
end_num = start_num + num
user_df = url_list[start_num:end_num]# 맨 마지막 크롤링할떄는 end_num을 지우세요. [start_num:] <-이렇게 
user_df = user_df.reset_index(drop=True)
user_df.head(3)

,검색어,아이디,url,담당
0,"삼성로119길 8, 37.5 청담",dOooOb,https://m.place.naver.com/my/5e4381768f87a842b...,지은
1,"논현로26길 40, 그로어스",HG9,https://m.place.naver.com/my/5f0614b46c567de86...,지은
2,"논현로26길 40, 그로어스",IAMA FOODIE,https://m.place.naver.com/my/5e1c41f88f87a842b...,지은



## 필요한 정보 

user_name  : 유저 아이디 o

place_name : 장소 이름 o

place_cate : 장소 카테고리 ㅇ

place_loc : 매장 주소 ㅇ

use_method : 이용 방법 ㅇ 

wait_time : 대기 시간 ㅇ

pupose : 목적 ㅇ

visit_with : 동행 ㅇ

place_review : 리뷰 내용 o

place_tag : 장소 태그 o -> 하다가 안됌.

visit_date : 방문 일자 -> o

visit_count : 방문 횟수 -> o





In [10]:
len(user_df)

29

In [11]:
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# driver = webdriver.Chrome(ChromeDriverManager().install()) 
from selenium import webdriver
driver = webdriver.Chrome('')
# # driver = webdriver.Chrome()4


scroll_num = 80
total_iterations = len(user_df)

place_name_list= []
user_name_list = []
place_cate_list = []
place_loc_list =[]
use_method_list =[]
wait_time_list =[]
purpose_list =[]
visit_with_list =[]
place_review_list =[]
place_tag_list =[]
visit_date_list =[]
visit_count_list =[]
visit_day_list =[]
visit_weekday_list =[]


for i in tqdm(range(total_iterations),desc='진행 중',position=0, leave= True):
    driver.get(user_df['url'][i])
    user_name = user_df['아이디'][i]
    time.sleep(5)
    
    # 사진/영상리뷰 체크박스 해제
    try:
        checkbox = driver.find_element(By.CLASS_NAME,'_9opr2y')
        # checkbox.click()
        driver.execute_script("arguments[0].click();", checkbox)
    except :
        pass 
    time.sleep(3)
    
    try :
        # 맨처음 게시물 클릭
        # first_post = driver.find_element(By.CLASS_NAME, 'pMmt8u')
        
        first_post = driver.find_element(By.CLASS_NAME, '_3gfKh3')
        # first_post.click()
        driver.execute_script("arguments[0].click();", first_post)
    except :
        pass
    time.sleep(3)
    
    for _ in range(scroll_num):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(8)
        try:
            buttons = driver.find_elements(By.XPATH, '//button[contains(.,"내용 더보기")]')
            if buttons:
                first_button = buttons[0]
                first_button.click()
                time.sleep(3)
        except :
            pass
        
        
        # try :   
        #     buttons2 = driver.find_elements(By.XPATH, '//span[contains(., "개의 리뷰 더 있습니다")]')
        #     buttons2 = driver.find_elements(By.CLASS_NAME, 'COw42b')
        #     if buttons2:
        #         first_button2 = buttons2[1]
        #         first_button2.click()
        #         time.sleep(1)
        # except :
        #     pass 
    time.sleep(3)
    user_review_elements = driver.find_elements(By.CLASS_NAME, 'ax747p.dfrH_x')

    for user_review in user_review_elements:
        
        # try:
        #     buttons = user_review.find_elements(By.XPATH, '//button[contains(.,"내용 더보기")]')
        #     if buttons:
        #         first_button = buttons[0]
        #         first_button.click()
        #         time.sleep(1)
        # except :
        #     pass
            
        time.sleep(2)
        try :   
            buttons2 = user_review.find_elements(By.XPATH, '//span[contains(., "개의 리뷰 더 있습니다")]')
            buttons2 = user_review.find_elements(By.CLASS_NAME, 'GnPRJ5')
            if buttons2:
                first_button2 = buttons2[1]
                first_button2.click()
                time.sleep(1)
        except :
            pass 
        # # 각 리뷰 요소 내에서 장소 이름을 찾음
        # 매장 이름
        place_name_element = user_review.find_element(By.CLASS_NAME, '_33frdK')
        place_name = place_name_element.text if place_name_element else "정보 없음"
        # 매장 카테고리 및 주소
        place_info = user_review.find_elements(By.CLASS_NAME,'e92Yfi')
        if len(place_info) >=2 :
            place_cate = place_info[0].text
            place_loc = place_info[1].text
            
        else : 
            place_cate = '정보없음'
            place_loc = '정보없음'


        # 매장 이용 세부정보
        
        use_method = '정보없음'
        wait_time = '정보없음'
        purpose = '정보없음'
        visit_with = '정보없음'
       
        details_info_elements = user_review.find_elements(By.CLASS_NAME, '_1ewTPX')
        if details_info_elements:
            for details_info_element in details_info_elements:
                details_text = details_info_element.text
                
                # 이용 방법이 포함되어 있는 경우
                if '이용 방법' in details_text:
                    use_method = details_text.split('이용 방법')[1].strip()
                
                # 대기 시간이 포함되어 있는 경우
                if '대기 시간' in details_text:
                    wait_time = details_text.split('대기 시간')[1].strip()
                
                # 목적이 포함되어 있는 경우
                if '목적' in details_text:
                    purpose = details_text.split('목적')[1].strip()
                
                # 동행이 포함되어 있는 경우
                if '동행' in details_text:
                    visit_with = details_text.split('동행')[1].strip()
        
        
        # 매장 리뷰
        place_reviews_elements = user_review.find_elements(By.CLASS_NAME,'_3n2arC')
        try :
            place_review = place_reviews_elements[0].text
            place_review = remove_special_characters(place_review)
        except :
            place_review = '정보없음'
            
        # # 매장 태그
        # place_tag_elements = user_review.find_element(By.CLASS_NAME, 'COw42b')
        # place_tag = list(map(lambda x: x.text, place_tag_elements))
        # place_tag = str(place_tag).replace('[', '').replace(']', '')   
        
        place_tag_elements = user_review.find_elements(By.CLASS_NAME, 'GnPRJ5')
        if place_tag_elements:
            place_tag = list(map(lambda x: x.text, place_tag_elements))
            place_tag = str(place_tag).replace('[', '').replace(']', '')
        else: 
            place_tag = '정보없음'

        # 방문 정보 (방문 일자, 재방문 이력)
        visit_info = user_review.find_elements(By.CLASS_NAME,'_4sHGsK')
        if len(visit_info) >= 2:
            if '별점' in visit_info[0].text:
                visit_date_text = visit_info[1].text
                visit_count_text = visit_info[2].text
            else :
                visit_date_text = visit_info[0].text
                visit_count_text = visit_info[1].text
                
            visit_date = visit_date_text.split('년')[1].strip()  # 월, 일 정보 추출
            # visit_weekday = visit_date_text.split('년')[0].strip()[-1:]  # 요일 정보 추출   
            visit_weekday =  visit_date[-3:]
            visit_count = visit_count_text.split('번째 방문')[0].strip()
        else:
           visit_date = '정보없음'
           visit_count = '정보없음'
           visit_weekday = '정보없음'

        place_name_list.append(place_name)
        user_name_list.append(user_name)
        place_cate_list.append(place_cate)
        place_loc_list.append(place_loc)
        use_method_list.append(use_method)
        wait_time_list.append(wait_time)
        purpose_list.append(purpose)
        visit_with_list.append(visit_with)
        place_review_list.append(place_review)
        place_tag_list.append(place_tag)
        visit_date_list.append(visit_date)
        visit_weekday_list.append(visit_weekday)
        visit_count_list.append(visit_count)

user_review_dic = {
    'user_name' : user_name_list,
    'place_name' : place_name_list,
     'place_name' : place_name_list,
    'place_cate' : place_cate_list,
    'place_loc' : place_loc_list,
    'use_method' : use_method_list,
    'wait_time' :  wait_time_list,
    'purpose' : purpose_list,
    'visit_with' : visit_with_list,
    'place_review' : place_review_list,
    'place_tag' : place_tag_list,
    'visit_date_list' :visit_date_list,
    'visit_weekday' : visit_weekday_list,
    'visit_count' : visit_count_list

}


driver.quit()


진행 중: 100%|██████████| 29/29 [5:42:40<00:00, 708.98s/it]  


In [12]:
user_review_df = pd.DataFrame(user_review_dic)

# 유저의 리뷰
user_review_df.to_excel(f'./data/4_3_naver_user_크롤링_user_df_{start_num}_{end_num}_최신_지은.xlsx', index=False)

In [13]:
# user_review_df.to_excel(f'./data/4_3_naver_user_review_df_test.xlsx', index=False)